In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import math

# 请自己编写由数据计算体积的公式，Please write a formula for calculating volume based on data.
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Parameter(torch.tensor(1., dtype=torch.float64, requires_grad=True), requires_grad=True)
        self.l2 = nn.Parameter(torch.tensor(1., dtype=torch.float64, requires_grad=True), requires_grad=True)

    def forward(self, x):
        return 2 * (1 - x) * self.l2 + 2 * x * self.l1


class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, c):
        self.x = x
        self.c = c

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.c[idx]


NUM_EPOCHS = 10000


def calculate_volume(df):
    x_t = torch.tensor(df['x'], dtype=torch.float64)
    c_t = torch.tensor(df['C'], dtype=torch.float64)

    dataset = Dataset(x_t, c_t)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

    net = Net()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(NUM_EPOCHS):
        for x, c in dataloader:
            optimizer.zero_grad()
            pred = net(x)
            loss = criterion(pred, c)
            loss.backward()
            optimizer.step()

        if epoch % 100 == 0:
            with torch.no_grad():
                pred = net(x_t)
                loss = criterion(pred, c_t)
                print(f'epoch {epoch}, loss: {loss}')

    a = net.l2.item()
    l = net.l1.item()
    print(a, l)
    return math.sqrt(3) * (a ** 2) / 12 * math.sqrt(l ** 2 - a ** 2 / 3)

In [ ]:
if __name__ == '__main__':
    #-------------读取训练集,训练集地址已经设定好，不用修改------------------#
    #-----Read the training set, the address of the training set has been set, and the following section does not need to be modified-------#
    train_path = "/bohr/train-wpxg/v1/C_train.csv"
    data_train = pd.read_csv(train_path)
    #--------------在训练集上开始进行模型回归--Start Solving on Training Set-------------#
    v_train = calculate_volume(data_train)

    #-------------读取测试集---------------#“DATA_PATH”是测试集加密后的环境变量，按照如下方式可以在提交后，系统评分时访问测试集，但是选手无法直接下载
    #----Read the testing set, “DATA_PATH” is an environment variable for the encrypted test set. After submission, you can access the test set for system scoring in the following manner, but the contestant cannot download it directly.-----#
    if os.environ.get('DATA_PATH'):
        DATA_PATH = os.environ.get("DATA_PATH") + "/"
    else:
        print("Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象")
        print("When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.")
        #Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象
        #When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.
    testA_path = DATA_PATH + "C_testA.csv"  #读取测试集A，read test set A
    data_testA = pd.read_csv(testA_path)
    testB_path = DATA_PATH + "C_testB.csv" #读取测试集B, read test set B
    data_testB = pd.read_csv(testB_path)

    #--------------在测试集上开始进行模型回归--Start Solving on Testing Set-------------#
    v_testA = calculate_volume(data_testA)
    v_testB = calculate_volume(data_testB)
    v = [v_train,v_testA,v_testB]
    #-----保存参数v到CSV文件到submission.csv, to save the volumes v to the .csv file-----#
    df_params = pd.DataFrame({'v': v})
    print(df_params)
    csv_file_path = 'submission.csv'
    df_params.to_csv(csv_file_path, index=False)